In [ ]:
'''
TODO:

- replace each pixel value with some scaled distance to nearest activated pixel
- maybe convert to vector graphics?
- second model to differentiate between closely related images
- play around w loss_fn, activation funcs, optimizers and hidden layer sizes
- threading to train many model at once for a kinda gridsearch for batch_size and epochs
'''

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
import numpy as np
import pandas as pd
from collections import Counter
from PIL import Image

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

y = np.asarray(train['label']).astype('int32').reshape((-1,1))
train = train.drop('label', axis=1)

answers = [a[0] for a in y]

In [4]:
class Model:
    def __init__(self, X, y, output_size, hidden_layers, activation='sigmoid', optimizer='adam',
                 hidden_size=None, loss_fn=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=('accuracy',)):
        self.X, self.y = X, y
        self.model = Sequential()
        
        size = len(X.iloc[0])
        hidden_size = hidden_size if hidden_size else 2 * size // 3 + output_size
        
        self.model.add(Dense(hidden_size, input_shape=(size,), activation=activation))
        for _ in range(hidden_layers - 1):
            self.model.add(Dense(hidden_size, activation=activation))
            
        self.model.add(Dense(output_size, activation=activation))
        
        self.model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)
        
    def fit(self, epochs=30, batch_size=10, verbose=1): self.model.fit(self.X, self.y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    def score(self): return self.model.hist.history['accuracy'][-1]
    
    def get_errors(self): return list(zip(*[(answers[i], a) for i, a in enumerate([a.argmax() for a in self.model.predict(train)]) if a != y[i]]))
    
    def get_threshold(self, a, b, predictions, boolean=False):
        error = [abs(x[a] - x[b]) for i, x in enumerate(predictions) if answers[i] in (a, b) and ((answers[i] == x.argmax()) == boolean)] 
        return sum(error) / len(error)
    
    def get_close(self, predictions, a, b, threshold=None):
        threshold = threshold if threshold else self.get_threshold(a, b, predictions)
        close = {i: train[i] for i, x in enumerate(predictions) if abs(x[a] - x[b]) < threshold}  # all inputs which were close
        for i, data in close.items():
            # use second model and save result, then edit predictions to have the new results
            continue
    
    def display_row(i): Image.fromarray(np.reshape(np.array(X.iloc[i].values), (28, 28)).astype('uint8')).show()
        
    def save_output(self, data, output='predictions.csv'):
        pd.DataFrame(columns=['ImageId', 'Label'], data=zip(range(1, len(data) + 1), [list(a).index(max(a)) for a in self.model.predict(data)])).to_csv(output, index=0)
        ! kaggle competitions submit -c digit-recognizer -f $output -m ""

In [8]:
model = Model(train, y, 10, 3, metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(epochs=50, batch_size=420000, verbose=1)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.4331 - sparse_categorical_accuracy: 0.0995
Epoch 2/50
1/1 [==============================] - 1s 563ms/step - loss: 2.3248 - sparse_categorical_accuracy: 0.1951
Epoch 3/50
1/1 [==============================] - 1s 622ms/step - loss: 2.3384 - sparse_categorical_accuracy: 0.1831
Epoch 4/50
1/1 [==============================] - 1s 592ms/step - loss: 2.2184 - sparse_categorical_accuracy: 0.2561
Epoch 5/50
1/1 [==============================] - 1s 597ms/step - loss: 2.0977 - sparse_categorical_accuracy: 0.5057
Epoch 6/50
1/1 [==============================] - 1s 600ms/step - loss: 2.0185 - sparse_categorical_accuracy: 0.4792
Epoch 7/50
1/1 [==============================] - 1s 623ms/step - loss: 1.9717 - sparse_categorical_accuracy: 0.2859
Epoch 8/50
1/1 [==============================] - 1s 573ms/step - loss: 1.9041 - sparse_categorical_accuracy: 0.2472
Epoch 9/50
1/1 [==============================] - 1s 607ms/step - l

In [ ]:
model.save_output(test)